In [ ]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, Cylinder, DiagramBuilder,
                         FindResourceOrThrow, GeometryInstance,
                         InverseKinematics, IrisInConfigurationSpace,
                         IrisOptions, JointSliders, LoadModelDirectives,
                         MakePhongIllustrationProperties, MeshcatVisualizer,
                         Parser, ProcessModelDirectives, RigidTransform,
                         RollPitchYaw, RotationMatrix, Solve, StartMeshcat)

from reproduction.util import *


# Helper methods (mostly from my class repos, these should move to Drake)
def AddTriad(source_id,
             frame_id,
             scene_graph,
             length=.25,
             radius=0.01,
             opacity=1.,
             X_FT=RigidTransform(),
             name="frame"):
    """
    Adds illustration geometry representing the coordinate frame, with the
    x-axis drawn in red, the y-axis in green and the z-axis in blue. The axes
    point in +x, +y and +z directions, respectively.

    Args:
      source_id: The source registered with SceneGraph.
      frame_id: A geometry::frame_id registered with scene_graph.
      scene_graph: The SceneGraph with which we will register the geometry.
      length: the length of each axis in meters.
      radius: the radius of each axis in meters.
      opacity: the opacity of the coordinate axes, between 0 and 1.
      X_FT: a RigidTransform from the triad frame T to the frame_id frame F
      name: the added geometry will have names name + " x-axis", etc.
    """
    # x-axis
    X_TG = RigidTransform(RotationMatrix.MakeYRotation(np.pi / 2),
                          [length / 2., 0, 0])
    geom = GeometryInstance(X_FT.multiply(X_TG), Cylinder(radius, length),
                            name + " x-axis")
    geom.set_illustration_properties(
        MakePhongIllustrationProperties([1, 0, 0, opacity]))
    scene_graph.RegisterGeometry(source_id, frame_id, geom)

    # y-axis
    X_TG = RigidTransform(RotationMatrix.MakeXRotation(np.pi / 2),
                          [0, length / 2., 0])
    geom = GeometryInstance(X_FT.multiply(X_TG), Cylinder(radius, length),
                            name + " y-axis")
    geom.set_illustration_properties(
        MakePhongIllustrationProperties([0, 1, 0, opacity]))
    scene_graph.RegisterGeometry(source_id, frame_id, geom)

    # z-axis
    X_TG = RigidTransform([0, 0, length / 2.])
    geom = GeometryInstance(X_FT.multiply(X_TG), Cylinder(radius, length),
                            name + " z-axis")
    geom.set_illustration_properties(
        MakePhongIllustrationProperties([0, 0, 1, opacity]))
    scene_graph.RegisterGeometry(source_id, frame_id, geom)


def AddMultibodyTriad(frame, scene_graph, length=.1, radius=0.005, opacity=1.):
    plant = frame.GetParentPlant()
    AddTriad(plant.get_source_id(),
             plant.GetBodyFrameIdOrThrow(frame.body().index()), scene_graph,
             length, radius, opacity, frame.GetFixedPoseInBodyFrame())


In [ ]:
meshcat = StartMeshcat()

In [ ]:

seed_points = {
    'nominal':
    [1.2, -0.7, 1.2, -2.1, -1.8, -0.1, -1.3, -2.5, -1.1, -1.3, 1.6, 0]
}

def parse_homecart(plant):
    parser = Parser(plant)
    parser.package_map().Add("gcs", GcsDir())
    directives = LoadModelDirectives(
        FindModelFile('models/homecart.yaml'))
    ProcessModelDirectives(directives, plant, parser)

def parse_sugarbox_scenario(plant):
    parse_homecart(plant)
    Parser(plant).AddModelFromFile(
        FindResourceOrThrow(
            'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    X_sugar = RigidTransform(RollPitchYaw(-np.pi/2, 0, 0.3), [0.45, 0.36, 0.11])
    plant.WeldFrames(plant.world_frame(),
                     plant.GetFrameByName('base_link_sugar'), X_sugar)


In [ ]:
def teleop_seeds(q0=None):
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_sugarbox_scenario(plant)
    #    parse_homecart(plant)
    #    Parser(plant).AddModelFromFile(
    #        FindResourceOrThrow(
    #            'drake/manipulation/models/ycb/sdf/004_sugar_box.sdf'))
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    sliders = builder.AddSystem(JointSliders(meshcat, plant, initial_value=q0))
    diagram = builder.Build()
    sliders.Run(diagram)
    # Print current value
    context = sliders.CreateDefaultContext()
    q = sliders.get_output_port().Eval(context)
    meshcat.DeleteAddedControls()
    print(f"q = {q}")
    return q

teleop_seeds(seed_points['nominal'])
#teleop_seeds()

In [ ]:
def make_ik_diagram():
    meshcat.Delete()
    meshcat.SetProperty('/Background', 'visible', False)
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_sugarbox_scenario(plant)
    left_grasp = plant.GetFrameByName("grasp_frame",
                             plant.GetModelInstanceByName("gripper_left"))
    AddMultibodyTriad(left_grasp, scene_graph)
    right_grasp = plant.GetFrameByName("grasp_frame",
                             plant.GetModelInstanceByName("gripper_right"))
    AddMultibodyTriad(right_grasp, scene_graph)
    sugar_frame = plant.GetFrameByName("base_link_sugar")
    AddMultibodyTriad(sugar_frame, scene_graph, length=0.15)
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)
    return diagram

ik_diagram = make_ik_diagram()

In [ ]:
def ik_grasp(diagram, q0):
    plant = diagram.GetSubsystemByName("plant")
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    grasp_frame = plant.GetFrameByName("grasp_frame",
                             plant.GetModelInstanceByName("gripper_left"))
    sugar_frame = plant.GetFrameByName("base_link_sugar")
    ik = InverseKinematics(plant, plant_context)
    ik.prog().AddQuadraticErrorCost(np.eye(len(q0)), q0, ik.q())
    # Center of grasp from is high center of the box.
    height_above_box_center = 0.025
    ik.AddPositionConstraint(grasp_frame, [0, 0, 0], sugar_frame,
                             [0, -height_above_box_center, 0],
                             [0, -height_above_box_center, 0])
    # Grasp is aligned with the box normals.
    ik.AddPositionConstraint(grasp_frame, [-0.1, 0, 0], sugar_frame,
                             [0, -height_above_box_center, 0.1],
                             [0, -height_above_box_center, 0.1])
    # Make sure the hand approaches from the top.
    ik.AddPositionConstraint(grasp_frame, [0, -0.1, 0], sugar_frame,
                             [-np.inf, -np.inf, -np.inf],
                             [np.inf, -height_above_box_center - 0.08, np.inf])
    #ik.AddMinimumDistanceConstraint(0.0)
    result = Solve(ik.prog())
    print(result.is_success())
    q_sol = result.GetSolution(ik.q())

    plant.SetPositions(plant_context, q_sol)
    diagram.Publish(context)

ik_grasp(ik_diagram, seed_points['nominal'])


In [ ]:
def ik_handover(diagram, q0):
    plant = diagram.GetSubsystemByName("plant")
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    left_grasp = plant.GetFrameByName("grasp_frame",
                             plant.GetModelInstanceByName("gripper_left"))
    right_grasp = plant.GetFrameByName(
        "grasp_frame", plant.GetModelInstanceByName("gripper_right"))
    ik = InverseKinematics(plant, plant_context)
    ik.prog().AddQuadraticErrorCost(np.eye(len(q0)), q0, ik.q())
    # center of right relative to left
    ik.AddPositionConstraint(left_grasp, [0, 0, 0], right_grasp,
                             [0, 0.05, 0],
                             [0, 0.2, 0])
    # x axes should align
    #ik.AddPositionConstraint(left_grasp, [1, 0, 0], right_grasp,
    #                         [-np.inf, 0, 0],
    #                         [np.inf, 0, 0])
    # y axes should be opposed
    #ik.AddPositionConstraint(left_grasp, [0, 1, 0], right_grasp,
    #                         [-.1, -np.inf, -.1],
    #                         [.1, 0, .1])
    # z axes should align
    ik.AddPositionConstraint(left_grasp, [0, 0, 1], right_grasp,
                             [0, 0, -np.inf],
                             [0, 0, np.inf])
    result = Solve(ik.prog())
    print(result.is_success())
    q_sol = result.GetSolution(ik.q())

    plant.SetPositions(plant_context, q_sol)
    diagram.Publish(context)

q = [1.75, -1.04, 1.27, -1.79, -2.75, 0.25, -1.45, -2.5, -1.04, -1.32, 3.11, 0]
ik_handover(ik_diagram, q)


In [ ]:
from pydrake.all import GlobalInverseKinematics

def ik_handover(diagram, q0):
    plant = diagram.GetSubsystemByName("plant")
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    left_gripper = plant.GetBodyByName(
        "body", plant.GetModelInstanceByName("gripper_left"))
    right_gripper = plant.GetBodyByName(
        "body", plant.GetModelInstanceByName("gripper_right"))
    ik = GlobalInverseKinematics(plant)
    ik.AddPostureCost(q0, plant.num_bodies()*[1], plant.num_bodies()*[1])
    # center of right relative to left
    # TODO(russt): Support frames
    #ik.AddWorldRelativePositionConstraint(right_gripper.index(), [0, .15, 0],
    #                                      left_gripper.index(), [0, .15, 0],
    #                                      [0, 0, 0], [0, 0, 0])
    ik.AddWorldPositionConstraint(left_gripper.index(), [0, 0, 0],
                                  [0.4, 0.2, 0.4], [0.4, 0.2, 0.4])
    result = Solve(ik.prog())
    print(result.is_success())
    q_sol = ik.ReconstructGeneralizedPositionSolution(result)

    plant.SetPositions(plant_context, q_sol)
    diagram.Publish(context)

q = [1.75, -1.04, 1.27, -1.79, -2.75, 0.25, -1.45, -2.5, -1.04, -1.32, 3.11, 0]
ik_handover(ik_diagram, q)

In [ ]:
def run_iris(seed):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parse_homecart(plant)
    plant.Finalize()
    diagram = builder.Build()

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    
    iris_options = IrisOptions()
    iris_options.require_sample_point_is_contained = True
    iris_options.iteration_limit = 5
    iris_options.termination_threshold = -1
    iris_options.relative_termination_threshold = 0.02
    iris_options.enable_ibex = False

    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Time:", np.round((time.time() - start_time)/60., 4),
          "minutes.\tFaces", len(hpoly.b()), flush=True)
    return hpoly

run_iris(seed_points['nominal'])